In [1]:
import os
repo_dir = os.environ.get("REPO_DIR")
code_dir = os.path.join(repo_dir, "code/")
data_dir = os.path.join(repo_dir, "data/")
os.chdir(code_dir)

import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import geopandas as gpd
import shapely
import fiona

from affine import Affine

import pandas as pd
import rasterio
import rasterio.mask
from rasterio import warp

import warnings

from nl_helpers import (apply_polygon_mask_and_return_flat_array, 
                        correct_nl_df_creation, bins,create_nl_binned_dataframe,
                       get_avg_nl_and_weighted_avg_nl,correct_nl_Ys)

# For NL downscaling experiment, we want to get mean lumninosity values of various polygons

### We need these for ADM2, ADM1, and ADM0 shapes. 

##### Since it's easy, we're going to get the pop weighted and the area weighted values

In [9]:
pop_adj_to_nl_outpath = (data_dir + "int/GPW_pop/"
           "gpw_v4_population_density_rev10_2015_30_sec_shifted_to_match_DMSP.tif")

dmsp_adj_to_pop_outpath = (data_dir + "int/DMSP_NL/"
           "DMSP_F182013.v4c_web.stable_lights.avg_vis_shifted_to_match_pop_raster.tif")

nl_adj = rasterio.open(dmsp_adj_to_pop_outpath)
pop_adj = rasterio.open(pop_adj_to_nl_outpath)

### ADM2 

In [ ]:
file = data_dir + "raw/geoBoundaries/geoBoundariesCGAZ_ADM2.geojson"
adm2 = gpd.read_file(file).set_index("shapeID").rename(columns = {"ADM1_shapeID": "ADM1_shape"})

In [ ]:
out_adm2 = get_avg_nl_and_weighted_avg_nl(adm2, raster_file=nl_adj, weight_raster =pop_adj)
out_adm2 = correct_nl_Ys(out_adm2, adm2, raster_file=nl_adj)

out_adm2.to_pickle(data_dir + "nl_downscaling/nl_Ys/dmsp_avg_nl_Ys_geoB_adm2.p")

### ADM1 -- from ADM2 geoBoundaries shapefile dissolved

In [ ]:
adm1_geoboundaries = adm2.dissolve("ADM1_shape")
out_adm1 = get_avg_nl_and_weighted_avg_nl(adm1_geoboundaries, raster_file=nl_adj, weight_raster =pop_adj)
out_adm1 = correct_nl_Ys(out_adm1, adm1_geoboundaries, raster_file=nl_adj)

out_adm1.to_pickle(data_dir + "nl_downscaling/nl_Ys/dmsp_avg_nl_Ys_geoB_adm1.p")

### ADM0 -- from ADM2 geoBoundaries shapefile dissolved

In [ ]:
adm0_geoboundaries = adm2.dissolve("shapeGroup")
out_adm0 = get_avg_nl_and_weighted_avg_nl(adm0_geoboundaries, raster_file=nl_adj, weight_raster =pop_adj)
out_adm0 = correct_nl_Ys(out_adm0, adm0_geoboundaries, raster_file=nl_adj)
out_adm0.to_pickle(data_dir + "nl_downscaling/nl_Ys/dmsp_avg_nl_Ys_geoB_adm0.p")
